# n-dimensional Gaussian process datasets 2

Fewer datasets, but more different numbers of training samples.

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0227-165745/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
ms_raw = pd.read_feather("measures.feather")

if "error" in ms_raw.columns:
    print("Errors:", len(ms_raw[~ms_raw["error"].isnull()]))
    ms_raw = ms_raw.loc[ms_raw["error"].isnull()]

ms_raw = smooth.analysis.expand_dataset_columns(ms_raw)
ms = ms_raw

# divergent_model_mask = (ms["loss"] == np.inf) | (~(ms["train_loss"] < 0.1))
# print("Divergent models:", len(ms[divergent_model_mask]))
# ms = ms.loc[~divergent_model_mask]

print("Remaining:", len(ms))
smooth.analysis.remove_constant_columns(ms, verbose=True)
# ms = smooth.analysis.expand_dataset_columns(ms)
ms.loc[:,"log_dir"] = ms["log_dir"].str.split("/").str.get(-1)

ms.head()

In [ ]:
ms["hidden_size"].nunique()

In [ ]:
for col in ms.columns:
    if ms[col].nunique() <= 20:
        sns.countplot(ms[col])
        plt.show()

# plt.hist(ms["dim"].astype(str))
# plt.bar

In [ ]:
cols = ["actual_epochs", "train_loss", "test_loss"]
cols = ms.columns

trim = 0.1

for col in cols:
    if ms[col].dtype == "object":
        continue
    
    data = ms.loc[(ms[col] >= ms[col].quantile(trim/2)) & (ms[col] <= ms[col].quantile(1-trim/2)), col]
    
    plt.hist(data, bins=20)
    plt.title(col)
    plt.show()

In [ ]:
ms.columns

In [ ]:
def get_optimal_path_length_f(dataset_name):
    dataset = smooth.datasets.from_name(dataset_name)
    n = len(dataset.x_test)
    y = sorted(dataset.y_test.reshape((-1,)))
#     cs = np.cumsum(y)[::-1]
#     res = 0
#     for i in range(n - 1):
#         res += cs[i] - y[i] * (n - i)

#     return res / (n ** 2)
    res = 0
    for a in y:
        for b in y:
            res += np.abs(a - b)
    return res / (n ** 2)

optimal_lengths = {}
for seed in tqdm.notebook.tqdm(range(1, 6)):
    optimal_lengths[seed] = get_optimal_path_length_f("gp-100-{}-1.0-77".format(seed))

In [ ]:
optimal_lengths

In [ ]:
ms1 = ms
measure_cols = ["gradient_norm", "path_length_d", "path_length_f"]
for measure in measure_cols + ["l2"]:
#     ax = plt.subplot()
#     ms1.loc[:,"hidden_size_s"] = ms1["hidden_size"].astype(str) + " units"
    sns.relplot(
        data=ms1,
        x="samples_train",
        y=measure,
        hue="hidden_size",
#         col="batch_size",
        kind="line",
#                 ax=ax
    )

In [ ]:
for measure in measure_cols:
    n_bins = 10
    bins = np.logspace(-2, 2, 20)
    plt.hist(ratios[measure], bins=np.linspace(-2, 2, 30))
#     plt.xscale("log")
    plt.title("log ratio of {}".format(measure))
    plt.show()
    
    print("Percentage of cases where ratio < 1: {:.1f}%".format(
        (ratios[measure] < np.log10(1)).sum() / len(ratios) * 100
    ))
    print("Percentage of cases where ratio < 1.5: {:.1f}%".format(
        (ratios[measure] < np.log10(1.5)).sum() / len(ratios) * 100
    ))
    print("90th percentile: ratio is {:.2f}".format(
        10 ** ratios[measure].quantile(0.9)
#         (ratios[measure] < np.log(1.5)).sum() / len(ratios) * 100
    ))
#     print(np.sum(ratios[measure]))
    

In [ ]:
np.logspace(np.log10(10), np.log10(1000), 10).round().astype(int)